In [ ]:
#import os
#os.getcwd()
#import sys
#sys.path.append('C:\\Users\\daniguvz\\OneDrive - Seguros Suramericana, S.A\\Documents\\GitHub\\pytrends\\pytrends')

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pytrends.request import TrendReq

In [3]:
from datetime import date

#se sefiene el intervalo de busqueda desde el 2004 hasta el momentio en que se ejecute este código
today = date.today()
Intervalo = ("2010-01-01 "+ str(today))

#importa los paquetes
from pytrends.request import TrendReq
import pandas as pd
from tqdm import tqdm
import time
import statsmodels.api as sm
import numpy as np


Paises = ['AR','BR','CL','MX','DO','SV','PA','UY','CO','PE'] # todos los países
#Paises= ['DO', 'AR', 'BO', 'BR', 'CL', 'CO', 'CR', 'EC', 'SV', 'GT', 'HN', 'MX', 'PY', 'PE', 'PA', 'UY', 'VE', 'IL'] #Paises con EMBI
#Paises = ['HT','DO'] # solo Haití y Dominicana

#buscar por el tema:
"""pytrends = TrendReq(hl='es', tz=360)
suggs = pytrends.suggestions("manifestación")
print(suggs)"""
#Query por a realizar
Query = ["/g/120ry1nd"]

frames = list()

#pytrends = TrendReq(hl='en-US', tz=360)
pytrends = TrendReq(hl='en-US', tz=360,   retries=2, backoff_factor=0.1, requests_args={'verify':False})


j = 0

#Se itera entre los países
for i in tqdm(Paises):
    
    #se hace el query
    pytrends.build_payload(Query, timeframe= Intervalo, geo= str(i)) #se construye el query
    data = pytrends.interest_over_time() # se extrae la linea de tiempo (interest_over_time) del query

    ####data = data.rename(columns={"/g/120ry1nd": 'protestas + manifestaciones'})
    data = data.reset_index()
    #Se normaliza
    #data[str(Query[0])] = data[str(Query[0])].div(Normalizacion[int(j)]) # Para paises sura
    data[str(Query[0])] = data[str(Query[0])].div(1) # Paises sin normalizacion

    # se apilan los resultados en una lista
    frames.append(data)  
    j = j+1
    
    time.sleep(30)
    
#se concatena las lista de resultados en un solo resultado    
result = pd.concat(frames, keys= Paises)
result = result.reset_index()
result.rename(columns={'level_0': 'Pais'}, inplace=True)


#Se hace el procesamiento y el filtro HP
lamnda = 60 #venta de filtro hp

resultados = list()
for i in Paises:
    data = result[result['Pais'] == str(i)] #el data no es el 
    Dataref = data.copy()
    #se reemplaza los 0 para no generar error con el log
    Dataref['log'] = data[str(Query[0])].replace(0, 0.01)
    #Se saca el logaritmo natural
    Dataref['log'] = np.log(Dataref['log'])
    #Se aplica el filtro
    Dataref['cycle'], Dataref['trend'] = sm.tsa.filters.hpfilter(Dataref['log'], lamnda)
    
    # se asigna el nivel de riesgo social
    
    #se puede comentar desde acá 
    
    """estado = []
    for i in Dataref['trend']:
        if i > 1.4:
            if i > 2:
                nivelriesgo = "Muy Alto"
            else:
                nivelriesgo = "Alto"
        else:
            nivelriesgo = "Bajo" 
        estado.append(nivelriesgo)
    Dataref['Estado del riesgo'] = estado"""
    #se puede comentar hasta acá 
    
    resultados.append(Dataref) 
    
    
Final = pd.concat(resultados, keys= Paises)


try:
    Final = Final.reset_index()
except:
    pass
Final.to_csv('TermometroRsocial.csv')

  0%|          | 0/10 [00:01<?, ?it/s]


RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/widgetdata/multiline?req=%7B%22time%22%3A+%222010-01-01+2023-07-24%22%2C+%22resolution%22%3A+%22MONTH%22%2C+%22locale%22%3A+%22en-US%22%2C+%22comparisonItem%22%3A+%5B%7B%22geo%22%3A+%7B%22country%22%3A+%22AR%22%7D%2C+%22complexKeywordsRestriction%22%3A+%7B%22keyword%22%3A+%5B%7B%22type%22%3A+%22ENTITY%22%2C+%22value%22%3A+%22%2Fg%2F120ry1nd%22%7D%5D%7D%7D%5D%2C+%22requestOptions%22%3A+%7B%22property%22%3A+%22%22%2C+%22backend%22%3A+%22IZG%22%2C+%22category%22%3A+0%7D%2C+%22userConfig%22%3A+%7B%22userType%22%3A+%22USER_TYPE_SCRAPER%22%7D%7D&token=APP6_UEAAAAAZL_HVbTW76vWkiaD-4KQonY9zFpYvizG&tz=360 (Caused by ResponseError('too many 429 error responses'))